In [3]:
!pip install -qq ax-platform

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ax 0.52.0 requires python-dateutil<=2.8.1, but you have python-dateutil 2.8.2 which is incompatible.
ax 0.52.0 requires SQLAlchemy<=1.3.13, but you have sqlalchemy 1.4.40 which is incompatible.


## Simple Function Optimisation

In [6]:
def booth(p): 
 # p = dictionary of parameters 
 x = (p["x1"] + (2 * p["x2"]) - 7)**2 + (2*p["x1"] + p["x2"] - 5)**2
 return x
print(booth({"x1":6, "x2": 7}))

365


In [7]:
from ax import optimize 
best_parameters, best_values, _, _ = optimize(parameters=[
    {"name": "x1",
    "type": "range",
    "bounds": [-10.0, 10.0],},
    {"name": "x2",
    "type": "range",
    "bounds": [-10.0, 10.0],},],
    evaluation_function=booth,
    minimize=True,)
print(best_parameters)

[INFO 03-03 22:10:49] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:10:49] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:10:49] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-10.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[-10.0, 10.0])], parameter_constraints=[]).
[INFO 03-03 22:10:49] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 03-03 22:10:49] ax.modelbridge.dispatch_uti

{'x1': 0.9559214649131054, 'x2': 3.2114451777646114}


/Users/jayanthboddu/miniforge3/envs/lightning-mps/lib/python3.8/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


## Parallel Optimisation Trials

In [10]:
# Running experiments in parallel 
from ax.service.ax_client import AxClient
ax = AxClient(enforce_sequential_optimization=False)
ax.create_experiment(
 name="booth_experiment",
 parameters=[
 {"name": "x1",
 "type": "range",
 "bounds": [-10.0, 10.0],},
 {"name": "x2",
 "type": "range",
 "bounds": [-10.0, 10.0],},],
 objective_name="booth",
 minimize=True,
)
for _ in range(100):
 next_parameters, trial_index = ax.get_next_trial()
 ax.complete_trial(trial_index=trial_index, raw_data=booth(next_parameters))
best_parameters, metrics = ax.get_best_parameters()



[INFO 03-03 22:15:52] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-03 22:15:52] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:15:52] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:15:52] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-10.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[-10.0, 10.0])], parameter_constraints=[]).
[INFO 03-03 22:

In [11]:
best_parameters

{'x1': 0.9937488946008806, 'x2': 3.0066547910255554}

## Hyper Parameter tuning a CNN on MNIST

In [12]:
import torch
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate, CNN

In [13]:
torch.manual_seed(12345)
dtype = torch.float
device = 'mps'

In [14]:
# Load MNIST data 
BATCH_SIZE = 512
train_loader, valid_loader, test_loader = load_mnist(batch_size=BATCH_SIZE)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/Users/jayanthboddu/miniforge3/envs/lightning-mps/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning:

The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1660136155850/work/torch/csrc/utils/tensor_numpy.cpp:178.)



In [15]:
# define a function to optimise 

def train_evaluate(parameterization):
    net = CNN()
    net = train(net=net, train_loader=train_loader, parameters=parameterization, dtype=dtype, device=device)
    return evaluate(
        net=net,
        data_loader=valid_loader,
        dtype=dtype,
        device=device,
    )

In [16]:
# run an optimisation loop 
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
    ],
    evaluation_function=train_evaluate,
    objective_name='accuracy',
)

[INFO 03-03 22:23:33] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:23:33] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter momentum. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-03 22:23:33] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-06, 0.4], log_scale=True), RangeParameter(name='momentum', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 03-03 22:23:33] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 03-03 22:23:33] ax.

In [17]:
best_parameters

{'lr': 0.00010782222631972221, 'momentum': 0.37680899742557505}

In [18]:
# plot response surface 
render(plot_contour(model=model, param_x='lr', param_y='momentum', metric_name='accuracy'))

In [19]:
# plot best objective as function of the iteration 
best_objectives = np.array([[trial.objective_mean*100 for trial in experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="Classification Accuracy, %",
)
render(best_objective_plot)